**Outline:**
- Build model and loss function
- Train model
- Observe valdidate
- Test

**To do:**
- Hyperparameter tuning
    + lr
    + layer1, layer2
    + betas

**Modification**
- Weight initialization with xavier uniform
- Adam optimization
- LR decay

**To do:**
- Add dropout
- Implement another loss function

In [1]:
import importlib
import argparse
import os, sys
import argparse
import pandas as pd
import numpy as np
import pickle
import time

In [2]:
import torch
import torch.utils.data
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler
from torch.autograd import Variable
from torchvision import transforms

In [3]:
sys.path.insert(0, './../utils/')
from utils import *
from models import *

In [4]:
#Define parser
#name = 'bpi_2012'
#name = 'bpi_2013'
name = 'small_log'
#name = 'large_log'

parser = {
    'train': True,
    'test': True,
    'model_class': 'LSTMAE',
    'model_name': ' ',
    'data_dir': '../data/',
    'data_file': name + '.csv',
    'nan_pct': 0.4,
    'input_dir': '../input/{}/'.format(name), 
    'batch_size' : 16,
    'epochs' : 20,
    'no_cuda' : False,
    'seed' : 7,
    'layer1': 25,
#    'layer2': 100,
#    'early_stopping': 30,
    'lr': 0.001,
    'betas': (0.9, 0.99),   
    'lr_decay': 0.99,
}

args = argparse.Namespace(**parser)
args.output_dir = './output/{0}_{1}_{2}/'.format(name, args.nan_pct, args.model_class)

In [5]:
if not os.path.isdir(args.output_dir):
    os.makedirs(args.output_dir)

In [6]:
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [7]:
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [8]:
kwargs = {'num_workers': 2, 'pin_memory': True} if args.cuda else {}

In [9]:
preprocessed_data_name = os.path.join(args.input_dir, 'preprocessed_data_{}.pkl'.format(args.nan_pct))
with open(preprocessed_data_name, 'rb') as f:
    min_max_storage = pickle.load(f)
    complete_matrix_w_normalized_time_train = pickle.load(f)
    missing_matrix_w_normalized_time_train = pickle.load(f)
    avai_matrix_train = pickle.load(f)
    nan_matrix_train = pickle.load(f)
    complete_matrix_w_normalized_time_val = pickle.load(f)
    missing_matrix_w_normalized_time_val = pickle.load(f)
    avai_matrix_val = pickle.load(f)
    nan_matrix_val = pickle.load(f)
    pad_matrix_val = pickle.load(f)
    complete_matrix_w_normalized_time_test = pickle.load(f)
    missing_matrix_w_normalized_time_test = pickle.load(f)
    avai_matrix_test = pickle.load(f)
    nan_matrix_test = pickle.load(f)
    pad_matrix_test = pickle.load(f)
    cols_w_time = pickle.load(f)
    cols_w_normalized_time = pickle.load(f)

In [10]:
file_name = os.path.join(args.input_dir, 'parameters_{}.pkl'.format(args.nan_pct))
with open(file_name, 'rb') as f:
    most_frequent_activity = pickle.load(f)
    first_timestamp = pickle.load(f)
    avai_instance = pickle.load(f)
    nan_instance = pickle.load(f)
    train_size = pickle.load(f)
    val_size = pickle.load(f)
    test_size = pickle.load(f)
    train_row_num = pickle.load(f)
    val_row_num = pickle.load(f)
    test_row_num = pickle.load(f)

# Load data

## Train

In [11]:
complete_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(complete_matrix_w_normalized_time_train, 
                                                                            batch_size=args.batch_size, shuffle=False, 
                                                                            num_workers=2)
missing_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(missing_matrix_w_normalized_time_train, 
                                                                           batch_size=args.batch_size, shuffle=False, 
                                                                           num_workers=2)
avai_matrix_trainLoader = torch.utils.data.DataLoader(avai_matrix_train, 
                                                      batch_size=args.batch_size, shuffle=False, 
                                                      num_workers=2)

In [12]:
complete_matrix_w_normalized_time_train.shape

(1200, 14, 15)

## Validate and test

In [13]:
normalized_complete_df_name = os.path.join(args.input_dir, 'normalized_complete_df_{}.csv'.format(args.nan_pct))
normalized_complete_df = pd.read_csv(normalized_complete_df_name)

normalized_missing_df_name = os.path.join(args.input_dir, 'normalized_missing_df_{}.csv'.format(args.nan_pct))
normalized_missing_df = pd.read_csv(normalized_missing_df_name)

In [14]:
missing_true_val = normalized_missing_df[train_row_num:-test_row_num].reset_index(drop=True)
complete_true_val = normalized_complete_df[train_row_num:-test_row_num].reset_index(drop=True)

In [15]:
missing_true_test = normalized_missing_df[-test_row_num:].reset_index(drop=True)
complete_true_test = normalized_complete_df[-test_row_num:].reset_index(drop=True)

In [16]:
missing_true_val.shape, missing_true_test.shape

((5600, 5), (5600, 5))

In [17]:
nan_time_index_val, nan_activity_index_val = getnanindex(missing_true_val)

In [18]:
nan_time_index_test, nan_activity_index_test = getnanindex(missing_true_test)

In [19]:
pd.isnull(normalized_missing_df).sum()

CaseID                   0
Activity             11178
CompleteTimestamp    11222
CumTimeInterval      11222
NormalizedTime       11222
dtype: int64

In [20]:
pd.isnull(missing_true_val).sum()

CaseID                  0
Activity             2230
CompleteTimestamp    2233
CumTimeInterval      2233
NormalizedTime       2233
dtype: int64

In [21]:
pd.isnull(missing_true_test).sum()

CaseID                  0
Activity             2222
CompleteTimestamp    2230
CumTimeInterval      2230
NormalizedTime       2230
dtype: int64

## Define model

In [22]:
if args.model_class == 'LSTMAE':
    model = LSTMAE(complete_matrix_w_normalized_time_train.shape[2], args.layer1, 1, args.cuda)
    
if args.cuda:
    model.cuda()

In [23]:
model.parameters

<bound method Module.parameters of LSTMAE (
  (encoder): EncoderRNN (
    (lstm): LSTM(15, 25, batch_first=True)
  )
  (decoder): DecoderRNN (
    (lstm): LSTM(25, 25, batch_first=True)
    (linear): Linear (25 -> 15)
    (sigmoid): Sigmoid ()
  )
)>

## Define loss

In [24]:
# Define loss
def loss_function(recon_x, x, avai_mask): 
    BCE = F.binary_cross_entropy(recon_x, x, weight=avai_mask, size_average=False)
    return BCE

In [25]:
optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=args.betas)

In [26]:
#Adjust learning rate per epoch: http://pytorch.org/docs/master/optim.html?highlight=adam#torch.optim.Adam

# Method 1:
lambda1 = lambda epoch: args.lr_decay ** epoch
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda1])

# Method 2:
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10)

## Utils

In [27]:
def save_model(model, epoch, score):
    model_file = os.path.join(args.output_dir, 'model_{}_epoch{}_score{:.4f}.pth'.format(args.model_class, epoch, score))
    torch.save(model.state_dict(), model_file)

In [28]:
def load_model(model, model_name):
    model_file = os.path.join(args.output_dir, model_name)
    assert os.path.isfile(model_file), 'Error: no model found!'
    model_state = torch.load(model_file)
    model.load_state_dict(model_state)

In [29]:
def val(model, missing_matrix_w_normalized_time_val, complete_matrix_w_normalized_time_val, avai_matrix_val):   
    model.eval()
    m_val = missing_matrix_w_normalized_time_val
    m_val = Variable(torch.Tensor(m_val).float())
    
    c_val = complete_matrix_w_normalized_time_val
    c_val = Variable(torch.Tensor(c_val).float())
    
    avai_matrix_val = Variable(torch.Tensor(avai_matrix_val).float())
    
    if args.cuda:
        m_val = m_val.cuda()
        c_val = c_val.cuda()
        avai_matrix_val = avai_matrix_val.cuda()

    recon_val = model(m_val)
    val_loss = loss_function(recon_val, c_val, avai_matrix_val)    
    
    return val_loss.data[0]/len(c_val.data)

In [30]:
missing_true_val.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,1201,Activity A,1970-01-01 09:00:00,0.0,0.000000
1,1201,Activity B,1970-01-01 10:00:00,3600.0,0.125000
2,1201,Activity C,1970-01-01 11:00:00,7200.0,0.249999
3,1201,Activity D,1970-01-01 12:00:00,10800.0,0.374999
4,1201,Activity E,NaN,NaN,NaN


In [31]:
complete_true_val.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,1201,Activity A,1970-01-01 09:00:00,0.0,0.000000
1,1201,Activity B,1970-01-01 10:00:00,3600.0,0.125000
2,1201,Activity C,1970-01-01 11:00:00,7200.0,0.249999
3,1201,Activity D,1970-01-01 12:00:00,10800.0,0.374999
4,1201,Activity E,1970-01-01 13:00:00,14400.0,0.499998


# Train model

In [32]:
def train(epoch, model, optimizer):
    model.train()
    train_loss = 0
    for batch_idx, (m_data, c_data, avai_mask) in enumerate(zip(missing_matrix_w_normalized_time_trainLoader,
                                                                complete_matrix_w_normalized_time_trainLoader,
                                                                avai_matrix_trainLoader)):
        
        c_data = Variable(c_data.float())
        m_data = Variable(m_data.float())
        avai_mask = Variable(avai_mask.float())
      
        if args.cuda:
            c_data = c_data.cuda()
            m_data = m_data.cuda()
            avai_mask = avai_mask.cuda()
            
        optimizer.zero_grad()
        
        recon_data = model(m_data)
        
        loss = loss_function(recon_data, c_data, avai_mask)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        
    return train_loss / len(complete_matrix_w_normalized_time_trainLoader.dataset)

In [33]:
if args.train:
    for epoch in range(1, args.epochs + 1):
        init = time.time()
        
        #method 1 scheduler
        scheduler.step()
        train_loss = train(epoch, model, optimizer)
        end_train = time.time()
        
        val_score = val(model, missing_matrix_w_normalized_time_val, 
                        complete_matrix_w_normalized_time_val, avai_matrix_val)
        
        end = time.time()
        print('====> Epoch {} | Train time: {:.4f} ms| End time: {:.4f} ms | Train loss: {:.4f} | Val score: {:.4f}'.
              format(epoch, (end_train-init)*1000, (end-init)*1000, train_loss, val_score))
else:
    load_model(model, args.model_name)

====> Epoch 1 | Train time: 485.9288 ms| End time: 505.2979 ms | Train loss: 57.1333 | Val score: 36.9004
====> Epoch 2 | Train time: 384.8050 ms| End time: 397.9373 ms | Train loss: 34.3661 | Val score: 32.6451
====> Epoch 3 | Train time: 397.2993 ms| End time: 415.8435 ms | Train loss: 31.0617 | Val score: 29.6238
====> Epoch 4 | Train time: 389.7729 ms| End time: 403.0042 ms | Train loss: 28.1745 | Val score: 26.8830
====> Epoch 5 | Train time: 383.4078 ms| End time: 402.1163 ms | Train loss: 25.5844 | Val score: 24.4464
====> Epoch 6 | Train time: 389.3311 ms| End time: 407.9618 ms | Train loss: 23.2575 | Val score: 22.1699
====> Epoch 7 | Train time: 384.6290 ms| End time: 403.3234 ms | Train loss: 21.0515 | Val score: 19.9769
====> Epoch 8 | Train time: 389.8399 ms| End time: 408.4096 ms | Train loss: 18.8894 | Val score: 17.8181
====> Epoch 9 | Train time: 395.7579 ms| End time: 414.7325 ms | Train loss: 16.7847 | Val score: 15.7496
====> Epoch 10 | Train time: 400.7292 ms| End 

# Predict and evaluate

In [34]:
if args.test:
    m_test = missing_matrix_w_normalized_time_test
    m_test = Variable(torch.Tensor(m_test).float())
    
    if args.cuda:
        m_test = m_test.cuda()
    
    print('Predicting...')
    recon_test = model(m_test)
    
    print('\n')
    print('Converting to dataframe...')
    recon_df_w_normalized_time = convert2df(recon_test, pad_matrix_test, cols_w_normalized_time, test_row_num)
    
    print('Transforming Normalized Time to Time...')
    recon_df_w_time = getDfWithTime(recon_df_w_normalized_time, missing_true_test, min_max_storage)
    
    print('Getting submission...')
    submission_df = getSubmission(recon_df_w_time, missing_true_test, complete_true_test, first_timestamp)
    submission = fixTime(submission_df)
        
    print('Testing...')
    mae_time, rmse_time, acc = evaluation(submission, nan_time_index_test, nan_activity_index_test,show=True)
    
    print('\n')
    print('Saving submission...')
    submission_df.to_csv(args.output_dir+'submission.csv', index=False)
    print('Done!')

Predicting...


Converting to dataframe...
Transforming Normalized Time to Time...
Getting submission...


/home/pllab/miniconda2/envs/pydata/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Testing...
Number of missing Time: 2230
Mean Absolute Error: 0.0256 day(s)
Root Mean Squared Error: 0.0385 day(s)
Number of missing Activity: 2222
Accuracy: 66.02%


Saving submission...
Done!


In [35]:
submission_df.head(20)

,CaseID,TrueActivity,PredictedActivity,TrueTime,PredictedTime,TrueCompleteTimestamp,PredictedCompleteTimestamp
0,1601,Activity A,Activity A,0.0,0.000000,1970-01-01 09:00:00,1970-01-01 09:00:00
1,1601,Activity B,Activity B,3600.0,3600.000000,1970-01-01 10:00:00,1970-01-01 10:00:00
2,1601,Activity C,Activity C,7200.0,7200.000000,1970-01-01 11:00:00,1970-01-01 11:00:00
3,1601,Activity D,Activity D,10800.0,10365.310420,1970-01-01 12:00:00,1970-01-01 11:52:45.310420
4,1601,Activity E,Activity E,14400.0,14699.277418,1970-01-01 13:00:00,1970-01-01 13:04:59.277418
5,1601,Activity G,Activity G,18000.0,17560.621582,1970-01-01 14:00:00,1970-01-01 13:52:40.621582
6,1601,Activity M,Activity M,18000.0,18000.000000,1970-01-01 14:00:00,1970-01-01 14:00:00
7,1601,Activity K,Activity K,18000.0,14390.060990,1970-01-01 14:00:00,1970-01-01 12:59:50.060990
8,1601,Activity I,Activity I,21600.0,21600.000000,1970-01-01 15:00:00,1970-01-01 15:00:00
9,1601,Activity J,Activity J,21600.0,22803.305267,1970-01-01 15:00:00,1970-01-01 15:20:03.305267


In [36]:
missing_true_test.head(20)

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,1601,Activity A,1970-01-01 09:00:00,0.0,0.000000
1,1601,Activity B,1970-01-01 10:00:00,3600.0,0.125000
2,1601,Activity C,1970-01-01 11:00:00,7200.0,0.249999
3,1601,Activity D,NaN,NaN,NaN
4,1601,Activity E,NaN,NaN,NaN
5,1601,Activity G,NaN,NaN,NaN
6,1601,Activity M,1970-01-01 14:00:00,18000.0,0.624998
7,1601,Activity K,NaN,NaN,NaN
8,1601,Activity I,1970-01-01 15:00:00,21600.0,0.749997
9,1601,Activity J,NaN,NaN,NaN
